# 1. Install Required Libraries
(Skip if already installed)

In [ ]:
# Uncomment below if needed:
# !pip install pandas openpyxl

# 2. Define Data Classes

In [ ]:

class Player:
    def __init__(self, name):
        self.name = name
        self.rounds = []  # List of PlayerRoundInfo

class PlayerRoundInfo:
    def __init__(self, player, tournament_name, round_number, handicap, tee, hole_scores, total, net):
        self.player = player
        self.tournament_name = tournament_name
        self.round_number = round_number
        self.handicap = handicap
        self.tee = tee
        self.hole_scores = hole_scores  # List of 18 scores
        self.total = total
        self.net = net

class Tournament:
    def __init__(self, name):
        self.name = name
        self.rounds = []  # List of Round

class Round:
    def __init__(self, tournament_name, round_number):
        self.tournament_name = tournament_name
        self.round_number = round_number
        self.player_rounds = []  # List of PlayerRoundInfo

class Team:
    def __init__(self, name, members):
        self.name = name
        self.members = members  # List of Players
        self.aggregate_score = None


# 3. Load Cleaned Excel Files

In [ ]:

import pandas as pd

# Update paths to where you saved your cleaned files
path_2023 = '/home/justin/JustInternetAI/Calcutta/Data/2023_cleaned_events_fixed_V2.xlsx'
path_2024 = '/home/justin/JustInternetAI/Calcutta/Data/2024_cleaned_events_V2.xlsx'

sheets_2023 = pd.ExcelFile(path_2023).sheet_names
sheets_2024 = pd.ExcelFile(path_2024).sheet_names


# 4. Build Player and Tournament Structures

In [ ]:

players = {}
tournaments = {}

def get_or_create_player(player_name):
    if player_name not in players:
        players[player_name] = Player(player_name)
    return players[player_name]

def load_event(file_path, sheet_name, year):
    df = pd.read_excel(file_path, sheet_name=sheet_name)

    tournament_name = f"{year} {sheet_name}"  # Full name including year
    tournament = Tournament(tournament_name)
    round_obj = Round(tournament_name, round_number=1)

    for _, row in df.iterrows():
        player = get_or_create_player(row['Player'])
        hole_scores = [row[i] for i in range(1, 19)]

        player_round = PlayerRoundInfo(
            player=player,
            tournament_name=tournament_name,
            round_number=1,
            handicap=row.get('Handicap'),
            tee=row['Tee'],
            hole_scores=hole_scores,
            total=row['Total'],
            net=row['Net']
        )

        player.rounds.append(player_round)
        round_obj.player_rounds.append(player_round)

    tournament.rounds.append(round_obj)
    tournaments[tournament_name] = tournament


# 5. Load All Events

In [ ]:

# Load all 2023 events
for sheet in sheets_2023:
    load_event(path_2023, sheet, year=2023)

# Load all 2024 events
for sheet in sheets_2024:
    load_event(path_2024, sheet, year=2024)


In [ ]:

# List all players
print(f"Loaded {len(players)} players.")
print(list(players.keys())[:10])  # Show first 10 players

# List all tournaments
print(f"Loaded {len(tournaments)} tournaments.")
print(list(tournaments.keys())[:5])

# Loop through all players and print their name and all rounds they played
for player_name, player_obj in players.items():
    print(f"\n{player_name} played {len(player_obj.rounds)} rounds:")
    for round_info in player_obj.rounds:
        print(f"- {round_info.tournament_name} (Net {round_info.net})")

# 6. Save Player and Tournaments to a file

In [ ]:


import pickle

# Save players and tournaments to a file
def save_data(players, tournaments, filename='golf_data.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump({'players': players, 'tournaments': tournaments}, f)

# Load players and tournaments from a file
def load_data(filename='golf_data.pkl'):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data['players'], data['tournaments']

In [ ]:
save_data(players, tournaments)
loaded_players, loaded_tournaments = load_data()
print(f"Loaded {len(loaded_players)} players from file.")